In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import glob

from matplotlib import pyplot as plt
import fitz
import cv2

## Excel table convert  to dataframe

In [ ]:
tables_paths=glob.glob('data/nov23/tables/*.xlsx')
data=[]
for path in tqdm(tables_paths):
    table=pd.read_excel(path)
    keys=[0,2,3,4,6,7,8]
    columns=np.array(list(table.columns))[keys]
    table=table[columns]
    table=table.values
    new_table=[]

    for i,line in enumerate(table):
        if not pd.isnull(line[0]):
            if '.pdf' in line[0]:
                line[0]=line[0][:-4]
            new_table.append(line)

    data.extend(new_table)

df=pd.DataFrame(data, columns=['id', 'surname', 'name', 'code', 'orthography','punctuation', 'grade'])

df['orthography']=pd.to_numeric(df['orthography'], downcast='integer', errors='coerce')
df['punctuation']=pd.to_numeric(df['punctuation'], downcast='integer', errors='coerce')

del df['grade']

df

In [ ]:
df.to_csv('gades_nov23.csv', index=False)

## Adding rule-based grades

In [ ]:
def marks2grade(orth, punc):
    grade=0

    if orth==0:
        if punc<=1:
            grade=5

        elif punc<=4:
            grade=4

        elif punc<=8:
            grade=3

    elif orth==1:
        if punc <=3:
            grade=4
        elif punc<=7:
            grade=3
    elif orth==2:
        if punc<=2:
            grade=4
        elif punc<=6:
            grade=3
    elif orth==3:
        if punc<=5:
            grade=3
    elif orth==4:
        if punc<=4:
            grade=3

    if orth>4 or (orth+punc)>8:
        grade=2

    return grade

In [ ]:
grades=pd.read_csv('gades_nov23.csv')

new_df=[]
for line in tqdm(df.values):
    gr=line[-1]
    grades_line=grades[grades.id==line[0]].values
    orth=line[4]
    punc=line[5]
    line=np.append(line, marks2grade(orth,punc))

    new_df.append(line)

new_df=pd.DataFrame(new_df, columns=['id', 'surname', 'name', 'code', 'orthography','punctuation', 'grade'])

new_df['aligned_doc']=None
new_df['type']='nov23'
new_df=new_df.dropna(subset=['orthography','punctuation'])
new_df.to_csv('grades_all.csv', index=False)
new_df

1) -1 это отсутствие оценки
2) 0 - статус "проверено" без оценки

In [ ]:
new_df['grade_true'].hist()
plt.show()